In [5]:
import pandas as pd
import json
from pandas import json_normalize

# Or relative path, does not work sometimes
#json_path = 'data\black_actors.csv'
data = json.load(open('D:\ITU\DataintheWild\Project\DITW_project-master\data\metadata\movies_metadata.json'))
data

{'1': {'title': 'Exodus: Gods and Kings',
  'url': 'https://en.wikipedia.org/wiki/Exodus%3A%20Gods%20and%20Kings',
  'year': '2014',
  'poster': 'https://upload.wikimedia.org/wikipedia/en/thumb/c/cd/Exodus2014Poster.jpg/220px-Exodus2014Poster.jpg',
  'actors': ['Christian Bale',
   'Joel Edgerton',
   'John Turturro',
   'Aaron Paul',
   'Ben Mendelsohn',
   'Sigourney Weaver',
   'Ben Kingsley'],
  'actors_urls': ['https://en.wikipedia.org/wiki/Christian_Bale',
   'https://en.wikipedia.org/wiki/Joel_Edgerton',
   'https://en.wikipedia.org/wiki/John_Turturro',
   'https://en.wikipedia.org/wiki/Aaron_Paul',
   'https://en.wikipedia.org/wiki/Ben_Mendelsohn',
   'https://en.wikipedia.org/wiki/Sigourney_Weaver',
   'https://en.wikipedia.org/wiki/Ben_Kingsley']},
 '2': {'title': 'The Little Mermaid (2023 film)',
  'url': 'https://en.wikipedia.org/wiki/The%20Little%20Mermaid%20%282023%20film%29',
  'year': '2023',
  'poster': 'https://upload.wikimedia.org/wikipedia/en/thumb/f/f4/The_Little_M

In [6]:
# Create dataframe from list of actors from json file

df = pd.DataFrame(
    [actor for v in data.values() for actor in v["actors"]], columns=["Actors"]
)
print(df)

                         Actors
0                Christian Bale
1                 Joel Edgerton
2                 John Turturro
3                    Aaron Paul
4                Ben Mendelsohn
5              Sigourney Weaver
6                  Ben Kingsley
7                  Halle Bailey
8              Jonah Hauer-King
9                  Daveed Diggs
10               Jacob Tremblay
11                    Awkwafina
12                    Art Malik
13               Noma Dumezweni
14                Javier Bardem
15             Melissa McCarthy
16           Scarlett Johansson
17               Takeshi Kitano
18                 Michael Pitt
19                  Pilou Asbæk
20  Chin Han (actor, born 1969)
21             Juliette Binoche
22               Bradley Cooper
23                   Emma Stone
24               Rachel McAdams
25                  Bill Murray
26               John Krasinski
27                Danny McBride
28                 Alec Baldwin
29                Jeremy Irvine
30      

In [7]:
# Create second dataframe, with colored actors

# relative path, comment the other one if wish to change
#black_actor_path = 'data/black_actors.csv' 

black_actor_path = 'D:/ITU/DataintheWild/Project/DITW_project-master/data/black_actors.csv'

df_actor_list = pd.read_csv(black_actor_path, on_bad_lines='skip')
df_actor_list.head()
print(df_actor_list)

                                                     id
0                                   Ernest Hogan – 1865
1     Matilda Sissieretta Joyner Jones (also known a...
2                                 George H. Reed – 1866
3                                       Bob Cole – 1868
4                              Madame Sul-Te-Wan – 1873
...                                                 ...
1045                                         Lydia West
1046                              Laura White (actress)
1047                                     Larissa Wilson
1048                                     Letitia Wright
1049                                      Angela Wynter

[1050 rows x 1 columns]


In [8]:
df.assign(indf=df.actor_name.isin(df_actor_list.id).astype(int))

AttributeError: 'DataFrame' object has no attribute 'actor_name'

,actor_name,actor_race
